In [5]:
from time import sleep, strftime
from random import randint
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import smtplib
from datetime import *
from __future__ import division, print_function  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

%matplotlib inline
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression

from sklearn.model_selection import (cross_val_score, train_test_split, 
                                     KFold, GridSearchCV)
import statsmodels.api as sm


In [2]:
chromedriver = "/Applications/chromedriver"

In [3]:
driver = webdriver.Chrome(chromedriver)

In [4]:
city_from = 'LNK'
city_to = 'ZRH'
date_start1 = '2021-12-12'
date_end1 = '2021-12-26'
date_start2 = '2021-12-19'
date_end2 = '2022-01-02'
date_start3 = '2022-01-16'
date_end3 = '2022-01-30'
date_start4 = '2021-11-07'
date_end4 = '2021-11-21'
date_start5 = '2021-11-14'
date_end5 = '2021-11-28'
date_start6 = '2021-12-05'
date_end6 = '2021-12-19'
date_start7 = '2022-01-02'
date_end7 = '2022-01-16'
date_start8 = '2022-01-09'
date_end8 = '2022-01-23'
date_start9 = '2021-10-10'
date_end9 = '2021-10-24'
date_start10 = '2021-10-17'
date_end10 = '2021-10-31'

date_start11 = '2022-01-23'
date_end11 = '2022-02-06'
date_start12 = '2022-01-30'
date_end12 = '2022-02-13'
date_start13 = '2022-02-06'
date_end13 = '2022-02-20'
date_start14 = '2022-02-13'
date_end14 = '2022-02-27'
date_start15 = '2022-02-20'
date_end15 = '2022-03-06'
date_start16 = '2022-02-27'
date_end16 = '2022-03-13'
date_start17 = '2022-03-06'
date_end17 = '2022-03-20'
date_start18 = '2022-03-13'
date_end18 = '2022-03-27'
date_start19 = '2022-03-20'
date_end19 = '2022-04-03'
date_start20 = '2022-03-27'
date_end20 = '2022-04-10'

date_start21 = '2022-04-03'
date_end21 = '2022-04-17'
date_start22 = '2022-04-10'
date_end22 = '2022-04-24'

In [5]:
def start_kayak(city_from, city_to, date_start, date_end):
    """City codes - it's the IATA codes!
    Date format -  YYYY-MM-DD"""
    kayak = ('https://www.kayak.com/flights/' + city_from + '-' + city_to +
             '/' + date_start + '-flexible/' + date_end + '-flexible?sort=bestflight_a')
    driver.get(kayak)
    sleep(30)
    
    df_flights_best = scrape()
    df_flights_best['sort'] = 'best'
    sleep(randint(60,80))
    
    print('switching to least expensive flights')
    kayak_inexpensive = ('https://www.kayak.com/flights/' + city_from + '-' + city_to +
             '/' + date_start + '-flexible/' + date_end + '-flexible?sort=price_a')
    driver.get(kayak_inexpensive)
    sleep(randint(60,90))
    
    
    df_flights_cheap = scrape()
    df_flights_cheap['sort'] = 'price'
    sleep(randint(60,80))
    
    print('switching to quickest flights')
    kayak_quick = ('https://www.kayak.com/flights/' + city_from + '-' + city_to +
             '/' + date_start + '-flexible/' + date_end + '-flexible?sort=duration_a')
    driver.get(kayak_quick)  
    sleep(randint(60,90))
    
    df_flights_fast = scrape()
    df_flights_fast['sort'] = 'fast'
    sleep(randint(60,80))
    
    final_df = df_flights_cheap.append(df_flights_best).append(df_flights_fast)
    final_df.to_csv('{}_flights_{}-{}_from_{}_to_{}.csv'.format(strftime("%Y%m%d-%H%M"),
                                                                                   city_from, city_to, 
                                                                                   date_start, date_end), index=False)
    print('Final DF Saved')

In [6]:
def load_more():
        more_results = driver.find_element_by_xpath('//a[contains(@id,"loadMore")]')
        more_results.click()
        print('waiting')
        sleep(randint(25,35))

In [7]:
def scrape():
    sleep(10)
    dates_a = driver.find_elements_by_xpath('//div[@class = "section date"]')
    time_a = driver.find_elements_by_xpath('//div[@class = "section times"]')
    num_stops_a = driver.find_elements_by_xpath('//div[@class = "section stops"]/div[1]')
    total_time_a = driver.find_elements_by_xpath('//div[contains(@class,"section duration")]')
    prices = driver.find_elements_by_xpath('//span[@class = "price-text"]')
    stop_cities = driver.find_elements_by_xpath('//div[@class = "section stops"]/div[2]')

    
    date_alist = []
    time_alist = []
    num_stops_alist = []
    total_time_alist = []
    price_lista = []
    stop_cities_list = []
    total_time_alist = []
    carrier_list = []
    for dd in range(len(dates_a)):
        date_alist.append(dates_a[dd].text)
        a_date_list = date_alist[::2]
        b_date_list = date_alist[1::2]
        a_day = [value.split()[0] for value in a_date_list]
        a_weekday = [value.split()[1] for value in a_date_list]
        b_day = [value.split()[0] for value in b_date_list]
        b_weekday = [value.split()[1] for value in b_date_list]
    for dt in range(len(time_a)):
        time_alist.append(time_a[dt].text)
        a_times_list = time_alist[::2]
        b_times_list = time_alist[1::2]
        a_times = [value.split('\n')[0] for value in a_times_list]
        a_depart = [value.split('–')[0] for value in a_times]
        a_arrive = [value.split('–')[1] for value in a_times]
        a_airline = [value.split('\n')[1] for value in a_times_list]
        b_times = [value.split('\n')[0] for value in b_times_list]
        b_depart = [value.split('–')[0] for value in b_times]
        b_arrive = [value.split('–')[1] for value in b_times]
        b_airline = [value.split('\n')[1] for value in b_times_list]
    for n in range(len(num_stops_a)):
        num_stops_alist.append(num_stops_a[n].text)
        a_num_stops = num_stops_alist[::2]
        b_num_stops = num_stops_alist[1::2]
    for n in range(len(stop_cities)):
        stop_cities_list.append(stop_cities[n].text)
        a_stop_cities = stop_cities_list[::2]
        b_stop_cities = stop_cities_list[1::2]
    for tt in range(len(total_time_a)):
        total_time_alist.append(total_time_a[tt].text)
        a_total_times = total_time_alist[::2]
        b_total_times = total_time_alist[1::2]
        a_time = [value.split('\n')[0] for value in a_total_times]
        b_time = [value.split('\n')[0] for value in b_total_times]
    for p in range(len(prices)):
        price_lista.append(prices[p].text)
        prices_list = [price.text.replace('$','') for price in prices if price.text != '']
        #prices_list = list(map(int, prices_list))
    


    load_more()

    cols = (['Out Day', 'Out Depart Time', 'Out Arrive Time', 'Out Weekday', 'Out Airline', 'Out Duration', 'Out Stops', 'Out Stop Cities',
            'Return Day', 'Return Depart Time', 'Return Arrive Time', 'Return Weekday', 'Return Airline', 'Return Duration', 'Return Stops', 'Return Stop Cities',
            'Price'])

    flights_df = pd.DataFrame({'Out Day': a_day,
                               'Out Weekday': a_weekday,
                               'Out Duration': a_time,
                               'Return Day': b_day,
                               'Return Weekday': b_weekday,
                               'Return Duration': b_time,
                               'Out Stops': a_num_stops,
                               'Out Stop Cities': a_stop_cities,
                               'Return Stops': b_num_stops,
                               'Return Stop Cities': b_stop_cities,
                               'Out Depart Time': a_depart,
                               'Out Arrive Time': a_arrive,
                               'Out Airline': a_airline,
                               'Return Depart Time': b_depart,
                               'Return Arrive Time':b_arrive,
                               'Return Airline': b_airline,
                               'Price': prices_list})[cols]



    flights_df['timestamp'] = strftime("%Y%m%d-%H%M")
    return flights_df

In [8]:
start_kayak(city_from, city_to, date_start1, date_end1)
start_kayak(city_from, city_to, date_start2, date_end2)
start_kayak(city_from, city_to, date_start3, date_end3)
start_kayak(city_from, city_to, date_start4, date_end4)
start_kayak(city_from, city_to, date_start5, date_end5)
start_kayak(city_from, city_to, date_start6, date_end6)
start_kayak(city_from, city_to, date_start7, date_end7)
start_kayak(city_from, city_to, date_start8, date_end8)
start_kayak(city_from, city_to, date_start9, date_end9)
start_kayak(city_from, city_to, date_start10, date_end10)
start_kayak(city_from, city_to, date_start11, date_end11)
start_kayak(city_from, city_to, date_start12, date_end12)
start_kayak(city_from, city_to, date_start13, date_end13)
start_kayak(city_from, city_to, date_start14, date_end14)
start_kayak(city_from, city_to, date_start15, date_end15)
start_kayak(city_from, city_to, date_start16, date_end16)
start_kayak(city_from, city_to, date_start17, date_end17)
start_kayak(city_from, city_to, date_start18, date_end18)
start_kayak(city_from, city_to, date_start19, date_end19)
start_kayak(city_from, city_to, date_start20, date_end20)
start_kayak(city_from, city_to, date_start21, date_end21)
start_kayak(city_from, city_to, date_start22, date_end22)

waiting
switching to least expensive flights
waiting
switching to quickest flights
waiting
Final DF Saved
waiting
switching to least expensive flights
waiting
switching to quickest flights
waiting
Final DF Saved
